In [1]:
import pandas as pd
import numpy as np
import matplotlib as mat 
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from scipy import stats
from datetime import datetime, date, time, timedelta

In [2]:
#ersetze in RET C und B mit nan
missing_values=["C","B","."]

In [4]:
crsp_m = pd.read_csv("./Data/Stockreturn_monthly_1960-2018.csv",na_values=missing_values)
#crsp_m = pd.read_csv("./Data/SIC CODE BANK.csv",na_values=missing_values)

crsp_m['date']=[datetime.strptime(str(date), "%Y%m%d")
for date in crsp_m['date']]

print(crsp_m.shape)

crsp_m.head()

(2434282, 7)


,PERMNO,date,SHRCD,EXCHCD,SICCD,PRC,RET
0,10001,1986-09-30,11,3,4920,6.375,-0.003077
1,10001,1986-10-31,11,3,4920,6.625,0.039216
2,10001,1986-11-28,11,3,4920,7.000,0.056604
3,10001,1986-12-31,11,3,4920,7.000,0.015000
4,10001,1987-01-30,11,3,4920,6.750,-0.035714


In [5]:
crsp_m[['PERMNO','SHRCD','EXCHCD']] = crsp_m[['PERMNO', 'SHRCD','EXCHCD']].astype(int)


In [6]:
crsp_m['RET']=crsp_m['RET'].fillna(0)

In [7]:
crsp_m['logret']=np.log(1+crsp_m['RET'])

In [8]:
crsp_m['logret+1']=crsp_m['logret']+1

In [9]:
crsp_m['MktV'] = crsp_m['SHRCD']*crsp_m['PRC']

In [10]:
#print (crsp_m)

In [11]:
#crsp_m.pivot(index='date',columns='PERMNO')

In [12]:
#piv1=crsp_m.pivot(index='date',columns='PERMNO',values= ['logret','MktV'])

In [13]:
pivm=crsp_m.pivot(index='date',columns='PERMNO',values= ['logret'])

In [14]:
#print(pivm)

In [15]:
CR_df = pivm

In [16]:
CR_df_columns = pivm.columns
CR_df_index = pivm.index.to_series()
CR_df = pd.DataFrame(index=CR_df_index, dtype=float, columns=CR_df_columns)

In [ ]:
for i in range(len(CR_df_index)):

    for j in range(len(CR_df_columns)):
        if np.sum(pivm.iloc[i-13:i,j].isna()) > 0:
            continue
        else:
            CR_df.iloc[i,j] = np.sum(pivm.iloc[i-13:i-2,j])

In [ ]:
print (CR_df)

In [ ]:
PT_df = CR_df.quantile([0.1, 0.9], axis=1).transpose()
PT_df.columns = ['Percentile 10', 'Percentile 90']

In [ ]:
#CR_df.to_csv(r'CR_df.csv')

In [ ]:
AC_df_columns = CR_df.columns
AC_df_index = CR_df.index.to_series()

In [ ]:
AC_df0 = pd.DataFrame(index=AC_df_index, dtype=int, columns=AC_df_columns)
AC_df1 = pd.DataFrame(index=AC_df_index, dtype=int, columns=AC_df_columns)
AC_df2 = pd.DataFrame(index=AC_df_index, dtype=int, columns=AC_df_columns)

In [ ]:
for i in range(len(AC_df_index)):

    for j in range(len(AC_df_columns)):
        if CR_df.iloc[i, j] <= PT_df.iloc[i, 0]:
             AC_df0.iloc[i, j] = True

        elif CR_df.iloc[i, j] <= PT_df.iloc[i, 1]:
             AC_df1.iloc[i, j] = True

        elif CR_df.iloc[i, j] > PT_df.iloc[i, 1]:
             AC_df2.iloc[i, j] = True

        else:
             continue

In [ ]:
AC_df0.fillna(value=False)
AC_df1.fillna(value=False)
AC_df2.fillna(value=False)

In [ ]:
# Descriptive Information of created Allocation0 DataFrame
#print(AC_df0.info())
#print(AC_df0.head(7))
#print(AC_df0.tail(7))
#print(AC_df0.shape)